In [1]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()


In [2]:

import numpy as np

# centro_replace = {
#         'C. O. ALG - ALGARVE': '8811455',
#         'C. O. AVEIRO': '8811417',
#         'C. O. BRAGA': '8817018',
#         'C. O. COIMBRA': '8818171',
#         'C. O. LEIRIA': '8811456',
#         'C. O. MEM MARTINS': '8811373',
#         'C. O. PERAFITA': '8818154',
#         'C. O. VISEU': '8811457',
#         'C.O. LOURES - MARL': '8811496',
#         'C.O. PALMELA': '8819009',
#         'CO DEVESAS': '8819001',
#         'CO PINHEIRO DE FORA': '8819002',
#     }

#selecional o centro e as datas:
    
CO='8818171'

import pandas as pd
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute("""
    SELECT * FROM recolhas_2023_fn 
    WHERE centro = %s 
    AND TO_DATE(data_criacao, 'DD/MM/YYYY') >= TO_DATE('03/11/2023', 'DD/MM/YYYY') 
    AND TO_DATE(data_criacao, 'DD/MM/YYYY') <= TO_DATE('30/11/2023', 'DD/MM/YYYY')
""", (CO,))

column_names = [desc[0] for desc in cur.description]
result_data_e = cur.fetchall()
cur.close()
con.close()

df_expresso = pd.DataFrame(result_data_e, columns=column_names)


In [3]:
one_drive_folder = "/Users/rubenfilipemartinsdarocha/Library/CloudStorage/GoogleDrive-rubenfilype@gmail.com/My Drive/Colab Notebooks/CTT/03.Modelo_parcial"
# Construct the full file path
file_name = str(CO) + '.xlsx'  # Replace CO with the appropriate variable name
file_path = os.path.join(one_drive_folder, file_name)

# Read Excel sheets
df_v = pd.read_excel(file_path, sheet_name='Veículos')
df = pd.DataFrame(df_v)


In [4]:
rotas_ficheiro_table = pd.DataFrame({'Nome': df['Nome'].str[:5].unique()})
rotas_expresso_table = pd.DataFrame({'giro': df_expresso['giro'].str[:5].unique()})

rotas_ficheiro_table['Nome'] = rotas_ficheiro_table['Nome'].astype(str).str.replace(' ', '')
rotas_expresso_table['giro'] = rotas_expresso_table['giro'].astype(str).str.replace(' ', '')

rotas_not_in_ficheiro = set(rotas_expresso_table['giro']) - set(rotas_ficheiro_table['Nome'])

pattern = r'^\d{4}[a-zA-Z]$'

# Filter routes matching the regular expression pattern
rotas_matching_pattern = [route for route in rotas_not_in_ficheiro if pd.Series(route).str.match(pattern).any()]

print("Routes in rotas_expresso_table not present in rotas_ficheiro_table and matching the pattern '4 numbers + 1 letter':")
print(rotas_matching_pattern)

Routes in rotas_expresso_table not present in rotas_ficheiro_table and matching the pattern '4 numbers + 1 letter':
[]
